# Import module

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

Or from the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

## ImageAnalysis3 and basic modules

In [21]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
from ImageAnalysis3 import *
from ImageAnalysis3.classes import _allowed_kwds

import h5py
import ast
import pandas as pd

print(os.getpid())

25920


## Chromatin_analysis_tools etc

See **functions** in the repository for [AnalysisTool_Chromatin](../../README.md)

In [22]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

# Define folders

In [23]:
# main folder for postanalysis
postanalysis_folder = r'L:\Shiwei\postanalysis_2024\v0'
# input files for postanalysis
input_folder = os.path.join(postanalysis_folder, 'resources_from_preprocess')

# output file to be generated
output_main_folder = os.path.join(postanalysis_folder, 'compartment_transcription')

output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

# make new folder if needed
make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')

Use existing analysis folder: L:\Shiwei\postanalysis_2024\v0\compartment_transcription\analysis.
Use existing figure folder: L:\Shiwei\postanalysis_2024\v0\compartment_transcription\figures.


# Plotting parameters

In [24]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
plt.rc('font', serif='Arial')

from ImageAnalysis3.figure_tools import _double_col_width, _single_col_width, _font_size, _ticklabel_size,_ticklabel_width

import seaborn as sns
sns.set_context("paper", rc={"font.size":_font_size,"axes.titlesize":_font_size+1,"axes.labelsize":_font_size})  

In [25]:
# Other required plotting parameters
_dpi = 300
_font_size = 7
_page_width = 5.5


## cell type color-codes

In [26]:
# cell labels from RNA-MERFISH and celltype prediction
selected_cell_labels = ['L2/3 IT','L4/5 IT','L5 IT','L6 IT','L5 ET','L5/6 NP','L6 CT','L6b',
                           'Sst','Pvalb','Lamp5','Sncg','Vip',
                           'Astro','Oligo','OPC','Micro','Endo','VLMC','SMC','Peri', 
                           #'other',
                          ]
# cell palette from RNA-MERFISH UMAP and stats
celltype_palette = {'Astro':'lightcoral', 
                    'Endo':'skyblue', 
                    'L2/3 IT':'gold', 
                    'L4/5 IT':'darkorange', 
                    'L5 ET':'mediumseagreen', 
                    'L5 IT':'aqua',
                    'L5/6 NP':'darkgreen',
                    'L6 CT':'brown',
                    'L6 IT':'magenta',
                    'L6b':'blue', 
                    'Lamp5':'orange', 
                    'Micro':'peachpuff',
                    'OPC':'thistle', 
                    'Oligo':'darkviolet',
                    'Peri':'sandybrown',
                    'Pvalb':'springgreen',
                    'SMC':'rosybrown',
                    'Sncg':'darkkhaki',
                    'Sst':'steelblue', 
                    'VLMC':'saddlebrown', 
                    'Vip':'red',
                    'other':'slategray'}


In [27]:
# this is the plotting order noted based on the snRNA transcriptional acitivty if needed
sorted_cellplot_order_byRNA = ['Micro', 'Oligo', 'Endo', 'OPC', 'Astro', 'Vip', 'Lamp5',
                  'L5/6 NP', 'Sst', 'Sncg', 'Pvalb', 'L4/5 IT', 'L6 CT',
                  'L6 IT', 'L6b', 'L2/3 IT', 'L5 IT', 'L5 ET']

# Load data relevant information

## load and format codebook

[merged codebook](../resources/merged_codebook.csv) as in the repository (merged for all DNA-MERFISH libraries)

In [28]:
# Load codebook 
codebook_fname = os.path.join(input_folder,'merged_codebook.csv')
codebook_df = pd.read_csv (codebook_fname, index_col=0)

# sort df by chr and chr_order
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)
codebook_df.head()

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_479,NDB_562,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604
0,1:3742742-3759944,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1:6245958-6258969,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1:8740008-8759916,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1016,1:9627926-9637875,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1017,1:9799472-9811359,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [29]:
# Format the chr loci name by 
# 1. changing loci name format
# 2. extract relevant information such as id, chr, chr_order, and library etc
from gene_to_loci import loci_pos_format
loci_name_list = list(map(loci_pos_format, codebook_df['name'].tolist()))
loci_name_arr = np.array(loci_name_list)

# convert to a new dataframe and set loci name as index
codebook_df = codebook_df[['name','id','chr','chr_order','library']]
codebook_df['loci_name'] = list(loci_name_arr[:,0])
codebook_df = codebook_df.set_index ('loci_name')

codebook_df.head()

,name,id,chr,chr_order,library
loci_name,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0.0,CTP11
chr1_6245958_6258969,1:6245958-6258969,2,1,1.0,CTP11
chr1_8740008_8759916,1:8740008-8759916,3,1,2.0,CTP11
chr1_9627926_9637875,1:9627926-9637875,1,1,3.0,CTP13
chr1_9799472_9811359,1:9799472-9811359,2,1,4.0,CTP13


# Load AB density ratio

Data can be generated from notebook 

[2_1_compute_ab_trans_density_singlecelll_majorType_by_pm](2_1_compute_ab_trans_density_singlecelll_majorType_by_pm.ipynb)

In [30]:
import pickle
gaussian_radius= 0.5
ABratio_ensemble_filename = os.path.join(output_analysis_folder, f'AB_trans_ratio_notNorm_r{gaussian_radius}_bymajorType_by_pm.pkl')
ABratio_ensemble_filename = os.path.join(output_analysis_folder, f'AB_trans_ratio_notNorm_r{gaussian_radius}_bymajorType_by_pm_backup.pkl')

if os.path.exists(ABratio_ensemble_filename):
    print(ABratio_ensemble_filename)
    AB_ratio_by_group = pickle.load(open(ABratio_ensemble_filename, 'rb'))

L:\Shiwei\postanalysis_2024\v0\compartment_transcription\analysis\AB_trans_ratio_notNorm_r0.5_bymajorType_by_pm_backup.pkl


In [31]:
AB_ratio_by_group.keys()

dict_keys(['Oligo', 'OPC', 'Micro', 'Astro', 'Endo', 'Peri', 'L2/3 IT', 'L4/5 IT', 'L5 IT', 'L6 IT', 'L5 ET', 'L6 CT', 'L5/6 NP', 'L6b', 'Vip', 'Pvalb', 'Lamp5', 'Sst', 'Sncg'])

# Calculate median ratio for all qualified cells

In [32]:
%matplotlib inline

import gene_activity
import loci_1d_features
from scipy import stats
import seaborn as sns
from tqdm import tqdm


calculate_ab = True
paralelle = True
num_threads = 16
ABratio_dict = {}


loci_key_list = loci_1d_features.sorted_loci_keys_for_loci_dataframe(codebook_df)
# ensure the chr_order is int not float for the processing below
loci_key_list_new = []
for _l in loci_key_list:
    loci_key_list_new.append((_l[0], int(_l[1])))         

if calculate_ab:
    if paralelle:
        import multiprocessing as mp
        _sel_class_list = list(AB_ratio_by_group.keys())
        mp_ABratio_list = []
        mp_class_list = []
        mp_key_list = []
        for _ind, sel_loci_ind in tqdm(enumerate(codebook_df.index.tolist()[:])):
            sel_loci_key = loci_key_list_new[_ind]
            mp_ABratio_list.append(AB_ratio_by_group)
            mp_class_list.append(_sel_class_list)
            mp_key_list.append(sel_loci_key)
            
        _start_time = time.time()
        print(f"-- summarize {len(mp_ABratio_list)} AB ratios with {num_threads} threads", end=' ')
        with mp.Pool(num_threads) as _summary_pool:
            all_AB_ratios_df_list = _summary_pool.starmap(
                                        loci_1d_features.sc_compartment_ratio_by_loci_key, 
                                        zip(mp_ABratio_list, mp_class_list, mp_key_list))
            _summary_pool.close()
            _summary_pool.join()
            _summary_pool.terminate()
        print(f"in {time.time()-_start_time:.3f}s.")
        all_ABratio_dfs = pd.concat(all_AB_ratios_df_list)
        all_ABratio_dfs.index = codebook_df.index[:]
        

    else:
        for _class in AB_ratio_by_group.keys():
            print(f'Process {_class}')
            ABratio_dict[_class] = {}
            for _ind, sel_loci_ind in tqdm(enumerate(codebook_df.index.tolist()[:])):
                sel_loci_key = loci_key_list_new[_ind]
                AB_ratios_df = loci_1d_features.sc_compartment_ratio_by_loci_key (AB_ratio_by_group, 
                                                                           [_class], 
                                                                           sel_loci_key, 
                                                                        average_ratios_in_cell=True, 
                                                                                       spot_num_th=600,
                                                                                       report_type = 'median')
                ABratio_dict[_class][_ind]=list(AB_ratios_df[_class])


1982it [00:00, 991899.60it/s]


-- summarize 1982 AB ratios with 16 threads in 803.254s.


## save the median AB ratio by type

In [35]:
if paralelle:
    ABratio_dfs = all_ABratio_dfs.copy(deep=True)
else:
    ABratio_df_list = []
    for _sel_class in ABratio_dict.keys():

        ABratio_df = pd.DataFrame.from_dict(ABratio_dict[_sel_class])
        ABratio_df_list.append(ABratio_df)

    ABratio_dfs = pd.concat(ABratio_df_list)
    if len(ABratio_dfs) == len(list(ABratio_dict.keys())):
        ABratio_dfs.index = list(ABratio_dict.keys())

    if len(ABratio_dfs.columns) == len(codebook_df.index):
        ABratio_dfs.columns = codebook_df.index
        
    ABratio_dfs = ABratio_dfs.transpose()

ABratio_dfs

,Oligo,OPC,Micro,Astro,Endo,Peri,L2/3 IT,L4/5 IT,L5 IT,L6 IT,L5 ET,L6 CT,L5/6 NP,L6b,Vip,Pvalb,Lamp5,Sst,Sncg
loci_name,,,,,,,,,,,,,,,,,,,
chr1_3742742_3759944,-0.430767,-0.420269,-0.255699,-0.453668,0.132135,-0.140469,-0.560620,-0.460501,-0.446779,-0.488172,-0.421529,-0.455810,-0.304009,-0.472670,-0.114805,-0.289297,-0.309272,-0.325902,-0.186869
chr1_6245958_6258969,-0.153712,-0.206244,-0.047506,-0.226636,0.387080,0.133823,-0.447946,-0.445162,-0.487868,-0.408086,-0.356980,-0.453940,-0.201882,-0.332559,-0.207538,-0.219555,-0.207729,-0.116463,-0.435847
chr1_8740008_8759916,-0.425727,-0.148089,-0.275335,-0.409980,0.080992,0.116621,-0.400666,-0.314289,-0.418091,-0.412230,-0.292372,-0.487285,-0.505100,-0.498226,-0.032649,-0.221845,-0.257861,-0.206280,-0.333840
chr1_9627926_9637875,-0.254296,0.003753,0.080439,-0.209209,0.287116,0.149472,-0.350141,-0.286409,-0.402433,-0.200084,-0.307043,-0.290176,-0.298747,-0.386660,-0.279638,-0.146983,-0.086232,-0.194100,-0.302644
chr1_9799472_9811359,-0.116000,-0.083814,0.234512,-0.073540,0.492773,0.359415,-0.246629,-0.303253,-0.354788,-0.248334,-0.232831,-0.253817,-0.417956,-0.329905,-0.355471,-0.403508,-0.072171,-0.268272,-0.355233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_166247682_166259932,0.190443,0.086648,0.045090,0.202411,0.315556,0.383854,-0.157234,-0.122200,-0.238115,-0.214688,-0.064618,-0.077107,-0.332221,-0.080294,-0.227362,-0.265453,-0.371283,-0.219211,-0.668820
chrX_167157164_167167452,0.069587,-0.185342,0.435091,0.024904,0.626812,0.500466,-0.203417,-0.119909,-0.157937,-0.201859,-0.364895,-0.162953,0.024588,-0.201043,-0.109136,-0.048074,-0.359983,-0.203461,-0.322641
chrX_168746045_168757590,-0.138497,-0.285900,-0.019425,-0.079555,0.332414,0.382330,-0.297829,-0.237427,-0.256210,-0.317645,-0.250351,-0.389821,0.137434,-0.329275,-0.261027,-0.155011,-0.287720,-0.211167,-0.682236


In [36]:
ABratio_dfs_fname = os.path.join(output_analysis_folder, r'Median_Summary_AB_trans_ratio_notNorm_r0.5_bymajorType_by_pm.csv')
ABratio_dfs.to_csv(ABratio_dfs_fname)